## Extract, Transform and Load (ETL) Step

We are going to use a custom CNN for object identification.

### Setup

In [13]:
import os
import random
import time
from dataclasses import dataclass
from pathlib import Path
from enum import Enum
import shutil
from tqdm import tqdm
import tensorflow as tf
import keras
import math
import numpy as np
from sklearn.utils import class_weight



ROOT_DIR = "datasets\\aircraft"
DATASET_DIR = "..\\dataset\\crop"

## Classes Mapping

We also need to provide a classes mapping, here is how we do that

In [14]:
CLASSES_RAW = [
    "a10",
    "a400m",
    "ag600",
    "ah64",
    "av8b",
    "an124",
    "an22",
    "an225",
    "an72",
    "b1",
    "b2",
    "b21",
    "b52",
    "be200",
    "c130",
    "c17",
    "c2",
    "c390",
    "c5",
    "ch47",
    "cl415",
    "e2",
    "e7",
    "ef2000",
    "f117",
    "f14",
    "f15",
    "f16",
    "f22",
    "f35",
    "f4",
    # IN DATA AS F18
    "f18",
    "h6",
    "j10",
    "j20",
    "jas39",
    "jf17",
    "jh7",
    "kc135",
    "kf21",
    "kj600",
    "ka27",
    "ka52",
    "mq9",
    "mi24",
    "mi26",
    "mi28",
    "mig29",
    "mig31",
    "mirage2000",
    "p3",
    "rq4",
    "rafale",
    "sr71",
    "su24",
    "su25",
    "su34",
    "su57",
    "tb001",
    "tb2",
    "tornado",
    "tu160",
    "tu22m",
    "tu95",
    "u2",
    "uh60",
    "us2",
    "v22",
    "vulcan",
    "wz7",
    "xb70",
    "y20",
    "yf23",
    "z19"
]

CLASSES = {model: i for i, model in enumerate(CLASSES_RAW)}

def get_class_id(class_str: str):
    return CLASSES[class_str.lower()]

print(CLASSES)

{'a10': 0, 'a400m': 1, 'ag600': 2, 'ah64': 3, 'av8b': 4, 'an124': 5, 'an22': 6, 'an225': 7, 'an72': 8, 'b1': 9, 'b2': 10, 'b21': 11, 'b52': 12, 'be200': 13, 'c130': 14, 'c17': 15, 'c2': 16, 'c390': 17, 'c5': 18, 'ch47': 19, 'cl415': 20, 'e2': 21, 'e7': 22, 'ef2000': 23, 'f117': 24, 'f14': 25, 'f15': 26, 'f16': 27, 'f22': 28, 'f35': 29, 'f4': 30, 'f18': 31, 'h6': 32, 'j10': 33, 'j20': 34, 'jas39': 35, 'jf17': 36, 'jh7': 37, 'kc135': 38, 'kf21': 39, 'kj600': 40, 'ka27': 41, 'ka52': 42, 'mq9': 43, 'mi24': 44, 'mi26': 45, 'mi28': 46, 'mig29': 47, 'mig31': 48, 'mirage2000': 49, 'p3': 50, 'rq4': 51, 'rafale': 52, 'sr71': 53, 'su24': 54, 'su25': 55, 'su34': 56, 'su57': 57, 'tb001': 58, 'tb2': 59, 'tornado': 60, 'tu160': 61, 'tu22m': 62, 'tu95': 63, 'u2': 64, 'uh60': 65, 'us2': 66, 'v22': 67, 'vulcan': 68, 'wz7': 69, 'xb70': 70, 'y20': 71, 'yf23': 72, 'z19': 73}


### Defining Dataset Constants

In [15]:
IMG_WIDTH = 256 # Keras default
IMG_HEIGHT = 256 # Keras default
IMG_CHANNELS = 3 # RGB

BATCH_SIZE = 32
EPOCHS = 10

### Extracting the Dataset

We need to first extract the x and y data from the dataset (image paths and labels).

### Splitting the Dataset

We need to split out dataset into three parts, train, validation and test.

We're going to use (80%, 10%, 10%) repartition for now.

In [16]:
def split_dataset(paths: list[str], classes: list[int], seed: int = None) -> tuple[list[str], list[int], list[str], list[int], list[str], list[int]]:
    x_train = []
    y_train = []
    x_val = []
    y_val = []
    x_test = []
    y_test = []

    if not seed:
        seed = time.time()

    random.seed(seed)

    for i in tqdm(range(len(paths))):
        rand = random.randint(1, 10)

        # Validation (10%)
        if rand == 9:
            x_val.append(paths[i])
            y_val.append(classes[i])

        # Test (10%)
        elif rand == 10:
            x_test.append(paths[i])
            y_test.append(classes[i])

        # Train (80%)
        else:
            x_train.append(paths[i])
            y_train.append(classes[i])

    return x_train, y_train, x_val, y_val, x_test, y_test

def extract_dataset(dataset_dir: str = DATASET_DIR, seed: int = None) -> None:
    aircraft_filepaths = []
    aircraft_classes = []
    for aircraft_dir in tqdm(os.listdir(dataset_dir)):
        aircraft_class = get_class_id(aircraft_dir)

        dir_path = os.path.join(dataset_dir, aircraft_dir)
        
        for aircraft_img in os.listdir(dir_path):
            aircraft_img_path = os.path.join(dir_path, aircraft_img)

            aircraft_filepaths.append(aircraft_img_path)
            aircraft_classes.append(aircraft_class)

    print(f"Found {len(aircraft_filepaths)} aircraft images")

    return aircraft_filepaths, aircraft_classes

paths, classes = extract_dataset()

x_train, y_train, x_val, y_val, x_test, y_test = split_dataset(paths, classes)

print(f"Train: {len(x_train)}")
print(f"Validation: {len(x_val)}")
print(f"Test: {len(x_test)}")

100%|██████████| 74/74 [00:00<00:00, 875.56it/s]


Found 31917 aircraft images


100%|██████████| 31917/31917 [00:00<00:00, 1450783.54it/s]

Train: 25598
Validation: 3163
Test: 3156


### Transforming the Dataset

We need to transform the dataset into a format that can be used by the model.

This includes

- Loading the images as tensors
- Resizing the images
- Normalizing the images (done later by the model)

### Lazy Loading and Tensorflow Datasets

We're going to use the `tf.data.Dataset` API to load the images lazily.

This is done to avoid loading all the images into RAM at once (10GB+ of images).

We also need to shuffle the dataset to avoid biasing the model since the images are ordered by class.

Notes:

- We're going to use a batch size of 32 for now
- We're going to use a prefetch buffer, automatically tuned by Tensorflow


In [17]:
class AircraftDataGenerator:
    def __init__(self, filepaths: list[str], classes: list[int], batch_size: int = BATCH_SIZE):
        self.filepaths = filepaths
        self.classes = classes
        self.batch_size = batch_size

    def load_image(self, filepath: str) -> tf.Tensor:
        image = tf.io.read_file(filepath)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])
        return image

    def create_dataset(self) -> tf.data.Dataset:
        dataset = tf.data.Dataset.from_tensor_slices((self.filepaths, self.classes))

        dataset = dataset.shuffle(buffer_size=len(self.filepaths), reshuffle_each_iteration=True)
        
        dataset = dataset.map(lambda x, y: (self.load_image(x), y), 
                            num_parallel_calls=tf.data.AUTOTUNE)
        
        
        dataset = dataset.batch(self.batch_size)

        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        return dataset
    
train_generator = AircraftDataGenerator(x_train, y_train)
val_generator = AircraftDataGenerator(x_val, y_val)
test_generator = AircraftDataGenerator(x_test, y_test)

train_dataset = train_generator.create_dataset()
val_dataset = val_generator.create_dataset()
test_dataset = test_generator.create_dataset()

### Building the Model



In [ ]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
    ]
)

model = keras.Sequential([
    data_augmentation,
    keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(),
    
    keras.layers.Conv2D(128, 3, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(),
    
    keras.layers.Conv2D(256, 3, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(CLASSES), activation='softmax')
])

y_train_np = np.array(y_train)
class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=np.unique(y_train_np),
                                                  y=y_train_np)
class_weights = dict(enumerate(class_weights))


optimizer = keras.optimizers.Adam(learning_rate=0.01)

def lr_decay(epoch):
  return 0.001 * math.pow(0.666, epoch)

lr_decay_callback = keras.callbacks.LearningRateScheduler(lr_decay, verbose=True)
early_stop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=True)
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath='best_model_sofar.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)


model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.summary()

model.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=[early_stop_callback, checkpoint_callback], class_weight=class_weights)

model.evaluate(test_dataset)

model.save("identification.keras")

print("Model saved")

c:\Users\ryana\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_6 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_3 (Rescaling)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.0265 - loss: 5.1697
Epoch 1: val_loss improved from inf to 4.22129, saving model to best_model_sofar.keras
800/800 ━━━━━━━━━━━━━━━━━━━━ 2093s 3s/step - accuracy: 0.0265 - loss: 5.1694 - val_accuracy: 0.0708 - val_loss: 4.2213 - learning_rate: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.000666.
Epoch 2/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.0490 - loss: 4.3723
Epoch 2: val_loss improved from 4.22129 to 3.99249, saving model to best_model_sofar.keras
800/800 ━━━━━━━━━━━━━━━━━━━━ 2082s 3s/step - accuracy: 0.0490 - loss: 4.3722 - val_accuracy: 0.0797 - val_loss: 3.9925 - learning_rate: 6.6600e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.00044355600000000006.
Epoch 3/10
749/800 ━━━━━━━━━━━━━━━━━━━━ 2:10 3s/step - accuracy: 0.0675 - loss: 4.0595

KeyboardInterrupt: 